# Preliminar Analysis

In [1]:
import requests
import time
import json
from urllib.parse import urlencode
import pandas as pd
import orjson
from tqdm import tqdm

### Download all articles where there are one or more french authors

In [ ]:
# BASE_URL = "https://api.openalex.org/works"

# selected_fields = ["doi", "publication_year", "language", "indexed_in", "primary_location", "best_oa_location", "open_access", "authorships", 
#                    "corresponding_author_ids", "corresponding_institution_ids", "apc_list", "apc_paid", "cited_by_count", "primary_topic", "topics", 
#                    "concepts", "grants", "awards", "funders"]

# def fetch_page(cursor, filters):
#     params = filters.copy()
#     params["cursor"] = cursor
#     params["per_page"] = 200
#     url = f"{BASE_URL}?{urlencode(params)}"
#     r = requests.get(url, timeout=30)
#     r.raise_for_status()
#     return r.json()

# for year in range(2013, 2025): # anar modificant l'initial year
#     print(f"\n=== YEAR {year} ===")

#     filters = {
#         "filter": ",".join([
#             "indexed_in:crossref",
#             "type:article|review",
#             # "primary_location.source.type:journal", # GUARANTEE THAT THEY ARE PUBLISHED IN JOURNALS
#             "authorships.institutions.country_code:FR",
#             f"from_publication_date:{year}-01-01",
#             f"to_publication_date:{year}-12-31",            
#         ]),
#         "select": ",".join(selected_fields)
#     }

#     cursor = "*"
#     count = 0
#     output_file = f"../data/interim/FranceInitialAPI/openalex_french_authors_{year}_v2.jsonl"

#     with open(output_file, "w", encoding="utf-8") as f:
#         pbar = tqdm(unit="works", dynamic_ncols=True)
#         while True:
#             data = fetch_page(cursor, filters)

#             works = data.get("results", [])
#             next_cursor = data.get("meta", {}).get("next_cursor", None)

#             for w in works:
#                 f.write(json.dumps(w) + "\n")
#                 count += 1

#             pbar.update(len(works))
            
#             if not next_cursor:
#                 break
#             cursor = next_cursor
#             time.sleep(1)

#         pbar.close()
#     print(f"\nSaved: {output_file}  (total {count})")

# print("\n=== COMPLETED ===")


=== YEAR 2023 ===


0works [00:00, ?works/s]

122504works [31:36, 64.58works/s]



Saved: ../data/interim/FranceInitialAPI/openalex_french_authors_2023_v2.jsonl  (total 122504)

=== YEAR 2024 ===


105076works [28:32, 61.34works/s]


Saved: ../data/interim/FranceInitialAPI/openalex_french_authors_2024_v2.jsonl  (total 105076)

=== COMPLETED ===


### Ingest BSO

In [24]:
interest = ['doi', 'bso_country_corrected', 'genre', 'openalex_id']
records = []
for year in tqdm(range(2013, 2025)):
    with open(f"../data/external/BSO/bso-publications-latest_split_{year}_enriched.jsonl", "rb") as f:
        for line in f:
            rec = orjson.loads(line)
            filtered = {k: rec.get(k) for k in interest} # Keep only the fields we care about
            if not filtered.get("doi"): # Skip records without DOI
                continue
            if 'fr' not in (filtered.get('bso_country_corrected') if isinstance(filtered.get('bso_country_corrected'), list) else [filtered.get('bso_country_corrected')] if filtered.get('bso_country_corrected') else []):
                continue
            if any(g in (filtered['genre'] if isinstance(filtered['genre'], list) else [filtered['genre']]) for g in ['journal-article', 'proceedings']):
                records.append(filtered)
                continue
   
df = pd.DataFrame(records)
df.to_csv('../data/interim/bso.csv')
df

100%|██████████| 12/12 [04:22<00:00, 21.84s/it]


,doi,bso_country_corrected,genre,openalex_id
0,10.1016/j.aforl.2013.06.122,"[other, fr]",journal-article,W2064129646
1,10.1016/j.rmr.2012.10.076,"[other, fr]",journal-article,W1966675032
2,10.4000/cem.13194,"[other, fr]",journal-article,None
3,10.1016/j.inoche.2013.04.002,"[fr, other]",journal-article,W2080941060
4,10.1016/j.nuclphysa.2013.02.010,"[fr, other]",journal-article,W2062167050
...,...,...,...,...
1726976,10.1093/nargab/lqae101,[fr],journal-article,W4401672078
1726977,10.1093/nargab/lqae064,[fr],journal-article,W4399656600
1726978,10.3917/vin.158.0087,[fr],journal-article,W4393062524
1726979,10.1186/s12913-024-10595-y,[fr],journal-article,W4391605821


In [2]:
interest = ['doi', 'publication_year', 'language', 'authorships', 'primary_location', 'topics', 'open_access', 'apc_list']
keys = ['doi', 'publication_year', 'language', 'field_names', 'journal', 'publisher', 'ins_type', '#_authors', 'oa_status', 'apc_list', 'corresponding', 'countries']
records = []
for year in tqdm(range(2013, 2025)):
    with open(f"../data/interim/FranceInitialAPI/openalex_french_authors_{year}_v2.jsonl", "rb") as f:
        for line in f:
            rec = orjson.loads(line)
            if not rec.get("doi"): # Skip records without DOI
                continue
            filtered = {k: rec.get(k) for k in interest} # Keep only the fields we care about

            # Extract field_names from topics
            topics = filtered.get("topics") or []
            filtered["field_names"] = {c.get("field", {}).get("display_name") for c in topics if c.get("field")}

            # Extract journal and publisher from primary_location
            pl = filtered.get("primary_location") or {}
            source = pl.get("source") or {}
            filtered["journal"] = source.get("display_name")
            filtered["publisher"] = source.get("host_organization_name")

            # Extract institution types and number of authors
            authorships = filtered.get("authorships") or []
            filtered["ins_type"] = {inst.get("type") 
                                    for auth in authorships 
                                    for inst in auth.get("institutions", []) if inst.get("type")}
            filtered["corresponding"] = [auth.get("is_corresponding") for auth in authorships]
            filtered["countries"] = [auth.get("countries") for auth in authorships]
            filtered["#_authors"] = len(authorships)
            
            o_a = filtered.get("open_access") or {}
            filtered["oa_status"] = o_a.get("oa_status")

            records.append({k: filtered.get(k) for k in keys})

df_oa = pd.DataFrame(records)
df_oa

100%|██████████| 12/12 [01:59<00:00,  9.98s/it]


,doi,publication_year,language,field_names,journal,publisher,ins_type,#_authors,oa_status,apc_list,corresponding,countries
0,https://doi.org/10.1016/j.cell.2013.05.039,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",Cell,Cell Press,"{healthcare, government, other, education, non...",5,bronze,"{'value': 10100, 'currency': 'USD', 'value_usd...","[False, False, False, True, False]","[[ES], [ES], [DE, GB], [ES], [FR]]"
1,https://doi.org/10.1051/0004-6361/201322068,2013,en,"{Physics and Astronomy, Computer Science}",Astronomy and Astrophysics,EDP Sciences,"{facility, nonprofit, government, education}",44,bronze,None,"[False, False, False, False, False, False, Fal...","[[DE], [US], [FR, US], [FR, US], [FR, US], [US..."
2,https://doi.org/10.1038/nature12477,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",Nature,Nature Portfolio,"{healthcare, government, other, education, non...",70,bronze,"{'value': 9750, 'currency': 'EUR', 'value_usd'...","[False, False, False, False, False, False, Fal...","[[GB], [GB], [GB], [CA], [GB], [GB], [GB], [GB..."
3,https://doi.org/10.1097/ccm.0b013e31827e83af,2013,en,{Medicine},Critical Care Medicine,Lippincott Williams & Wilkins,"{healthcare, company, education}",23,closed,None,"[False, False, False, False, False, False, Fal...","[[US], [US], [GB], [FR], [DE], [US], [US], [IL..."
4,https://doi.org/10.1083/jcb.201211138,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",The Journal of Cell Biology,Rockefeller University Press,"{nonprofit, government, education}",2,bronze,None,"[False, False]","[[FR], [NL]]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1427183,https://doi.org/10.1055/s-0043-1777875,2024,de,{},physioscience,Georg Thieme Verlag,"{healthcare, education}",4,closed,None,"[True, False, False, False]","[[DE], [DE], [FR], [DE]]"
1427184,https://doi.org/10.1055/s-0044-1786882,2024,de,{},Zeitschrift für Gastroenterologie,Thieme Medical Publishers (Germany),"{healthcare, nonprofit, education}",50,closed,None,"[True, False, False, False, False, False, Fals...","[[AT], [], [AT], [CH], [DE], [HR], [CN], [], [..."
1427185,https://doi.org/10.1055/s-0044-1791345,2024,en,{},Veterinary and Comparative Orthopaedics and Tr...,Schattauer Verlag,{education},3,closed,None,"[True, False, False]","[[FR], [FR], [FR]]"
1427186,https://doi.org/10.4000/11ndt,2024,fr,{Social Sciences},Histoire Politique,None,{facility},1,bronze,None,[True],[[FR]]


In [3]:
df = pd.read_csv('../data/interim/bso.csv')

df_oa['doi_corr'] = df_oa['doi'].apply(lambda x: x[16::])
df_oa_comb = df_oa.merge(df, left_on = 'doi_corr', right_on = 'doi', how = 'left')
df_oa_comb['BSO'] = df_oa_comb['openalex_id'].apply(lambda x: False if pd.isna(x) else True)
df_oa_comb = df_oa_comb.drop(columns = ['doi_y', 'bso_country_corrected', 'genre', 'openalex_id']).rename(columns = {'doi_x' : 'doi'})
df_oa_comb

,doi,publication_year,language,field_names,journal,publisher,ins_type,#_authors,oa_status,apc_list,corresponding,countries,doi_corr,Unnamed: 0,BSO
0,https://doi.org/10.1016/j.cell.2013.05.039,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",Cell,Cell Press,"{healthcare, government, other, education, non...",5,bronze,"{'value': 10100, 'currency': 'USD', 'value_usd...","[False, False, False, True, False]","[[ES], [ES], [DE, GB], [ES], [FR]]",10.1016/j.cell.2013.05.039,21968.0,True
1,https://doi.org/10.1051/0004-6361/201322068,2013,en,"{Physics and Astronomy, Computer Science}",Astronomy and Astrophysics,EDP Sciences,"{facility, nonprofit, government, education}",44,bronze,None,"[False, False, False, False, False, False, Fal...","[[DE], [US], [FR, US], [FR, US], [FR, US], [US...",10.1051/0004-6361/201322068,17753.0,True
2,https://doi.org/10.1038/nature12477,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",Nature,Nature Portfolio,"{healthcare, government, other, education, non...",70,bronze,"{'value': 9750, 'currency': 'EUR', 'value_usd'...","[False, False, False, False, False, False, Fal...","[[GB], [GB], [GB], [CA], [GB], [GB], [GB], [GB...",10.1038/nature12477,53871.0,True
3,https://doi.org/10.1097/ccm.0b013e31827e83af,2013,en,{Medicine},Critical Care Medicine,Lippincott Williams & Wilkins,"{healthcare, company, education}",23,closed,None,"[False, False, False, False, False, False, Fal...","[[US], [US], [GB], [FR], [DE], [US], [US], [IL...",10.1097/ccm.0b013e31827e83af,NaN,False
4,https://doi.org/10.1083/jcb.201211138,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",The Journal of Cell Biology,Rockefeller University Press,"{nonprofit, government, education}",2,bronze,None,"[False, False]","[[FR], [NL]]",10.1083/jcb.201211138,106587.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427243,https://doi.org/10.1055/s-0043-1777875,2024,de,{},physioscience,Georg Thieme Verlag,"{healthcare, education}",4,closed,None,"[True, False, False, False]","[[DE], [DE], [FR], [DE]]",10.1055/s-0043-1777875,NaN,False
1427244,https://doi.org/10.1055/s-0044-1786882,2024,de,{},Zeitschrift für Gastroenterologie,Thieme Medical Publishers (Germany),"{healthcare, nonprofit, education}",50,closed,None,"[True, False, False, False, False, False, Fals...","[[AT], [], [AT], [CH], [DE], [HR], [CN], [], [...",10.1055/s-0044-1786882,NaN,False
1427245,https://doi.org/10.1055/s-0044-1791345,2024,en,{},Veterinary and Comparative Orthopaedics and Tr...,Schattauer Verlag,{education},3,closed,None,"[True, False, False]","[[FR], [FR], [FR]]",10.1055/s-0044-1791345,NaN,False
1427246,https://doi.org/10.4000/11ndt,2024,fr,{Social Sciences},Histoire Politique,None,{facility},1,bronze,None,[True],[[FR]],10.4000/11ndt,NaN,False


**Download missing and filter for the wanted document type**

In [29]:
df_bso_oa = df[~df.openalex_id.isna()]
missing = df_bso_oa[~df_bso_oa.doi.isin(df_oa_comb.doi_corr)].doi.unique()

BASE_URL = "https://api.openalex.org/works"

selected_fields = ["doi", "publication_year", "language", "indexed_in", "primary_location", "best_oa_location", "open_access", "authorships", 
                   "corresponding_author_ids", "corresponding_institution_ids", "apc_list", "apc_paid", "cited_by_count", "primary_topic", "topics", 
                   "concepts", "grants", "awards", "funders"]

def fetch_by_ids(id_list):
    # Normalize IDs to a string separated by |
    doi_filter = "doi:" + "|".join(id_list)

    filters = [
        doi_filter,
        "indexed_in:crossref",
        "type:article|review",
        "primary_location.source.type:journal"
    ]

    params = {
        "filter": ",".join(filters),
        "select": ",".join(selected_fields),
        "per_page": 200
    }

    url = f"{BASE_URL}?{urlencode(params)}"
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        data = r.json()
        return data.get("results", [])
    except requests.exceptions.JSONDecodeError:
        print("JSON decode error for URL:", url[:200])
        pass
    except requests.exceptions.RequestException as e:
        print("Request error:", e)
        pass

batch_size = 50  # try increasing, test carefully
all_fetched = []
for i in tqdm(range(0, len(missing), batch_size)):
    batch = missing[i:i+batch_size]
    results = fetch_by_ids(batch)
    all_fetched.extend(results)

    
output_file = "../data/interim/FranceInitialAPI/openalex_missing.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for w in all_fetched:
        f.write(json.dumps(w) + "\n")

print(f"Saved {len(all_fetched)} records to {output_file}")

100%|██████████| 3644/3644 [20:58<00:00,  2.90it/s] 


Saved 10653 records to ../data/interim/FranceInitialAPI/openalex_missing.jsonl


In [4]:
interest = ['doi', 'publication_year', 'language', 'authorships', 'primary_location', 'topics', 'open_access', 'apc_list']
keys = ['doi', 'publication_year', 'language', 'field_names', 'journal', 'publisher', 'ins_type', '#_authors', 'oa_status', 'apc_list', 'corresponding', 'countries']
records = []

with open(f"../data/interim/FranceInitialAPI/openalex_missing.jsonl", "rb") as f:
    for line in f:
        rec = orjson.loads(line)
        if not rec.get("doi"): # Skip records without DOI
            continue
        filtered = {k: rec.get(k) for k in interest} # Keep only the fields we care about

        # Extract field_names from topics
        topics = filtered.get("topics") or []
        filtered["field_names"] = {c.get("field", {}).get("display_name") for c in topics if c.get("field")}

        # Extract journal and publisher from primary_location
        pl = filtered.get("primary_location") or {}
        source = pl.get("source") or {}
        filtered["journal"] = source.get("display_name")
        filtered["publisher"] = source.get("host_organization_name")

        # Extract institution types and number of authors
        authorships = filtered.get("authorships") or []
        filtered["ins_type"] = {inst.get("type") 
                                for auth in authorships 
                                for inst in auth.get("institutions", []) if inst.get("type")}
        filtered["corresponding"] = [auth.get("is_corresponding") for auth in authorships]
        filtered["countries"] = [auth.get("countries") for auth in authorships]
        filtered["#_authors"] = len(authorships)
            
        o_a = filtered.get("open_access") or {}
        filtered["oa_status"] = o_a.get("oa_status")

        records.append({k: filtered.get(k) for k in keys})

df_oa_missing = pd.DataFrame(records)
df_oa_missing

,doi,publication_year,language,field_names,journal,publisher,ins_type,#_authors,oa_status,apc_list,corresponding,countries
0,https://doi.org/10.3906/mat-1108-18,2013,en,"{Social Sciences, Economics, Econometrics and ...",TURKISH JOURNAL OF MATHEMATICS,Scientific and Technological Research Council ...,{},2,bronze,None,"[False, False]","[[], []]"
1,https://doi.org/10.3917/rhmc.601.0192,2013,fr,{Arts and Humanities},Revue d’histoire moderne et contemporaine,None,{},1,closed,None,[True],[[]]
2,https://doi.org/10.4000/lirico.1171,2013,es,"{Arts and Humanities, Social Sciences}",Cuadernos LIRICO,Réseau Interuniversitaire d'Ètude des Littérat...,{},1,diamond,None,[True],[[]]
3,https://doi.org/10.1088/0004-637x/772/1/7,2013,en,{Physics and Astronomy},The Astrophysical Journal,IOP Publishing,{},4,bronze,"{'value': 4499, 'currency': 'USD', 'value_usd'...","[True, False, False, False]","[[], [], [], []]"
4,https://doi.org/10.1016/j.datak.2013.06.002,2013,en,{Computer Science},Data & Knowledge Engineering,Elsevier BV,{},4,green,"{'value': 2590, 'currency': 'USD', 'value_usd'...","[False, False, False, False]","[[], [], [], []]"
...,...,...,...,...,...,...,...,...,...,...,...,...
10648,https://doi.org/10.1109/tcpmt.2024.3370951,2024,en,{Engineering},IEEE Transactions on Components Packaging and ...,Institute of Electrical and Electronics Engineers,{education},4,closed,None,"[False, False, False, False]","[[US], [], [DK], [DK]]"
10649,https://doi.org/10.1016/j.ymssp.2024.111486,2024,en,{Engineering},Mechanical Systems and Signal Processing,Elsevier BV,{facility},5,hybrid,"{'value': 4830, 'currency': 'USD', 'value_usd'...","[False, False, False, False, False]","[[], [], [], [], []]"
10650,https://doi.org/10.1016/j.finel.2024.104163,2024,en,{Engineering},Finite Elements in Analysis and Design,Elsevier BV,{facility},3,closed,"{'value': 3080, 'currency': 'USD', 'value_usd'...","[False, False, False]","[[], [], []]"
10651,https://doi.org/10.54563/cahiers-duras.519,2024,fr,"{Arts and Humanities, Psychology}",Cahiers Marguerite Duras.,None,{},1,green,None,[True],[[]]


In [ ]:
df_oa_missing['doi_corr'] = df_oa_missing['doi'].apply(lambda x: x[16::])
df_oa_comb_2 = df_oa_missing.merge(df, left_on = 'doi_corr', right_on = 'doi', how = 'left')
df_oa_comb_2['BSO'] = df_oa_comb_2['openalex_id'].apply(lambda x: False if pd.isna(x) else True)
df_oa_comb_2 = df_oa_comb_2.drop(columns = ['doi_y', 'bso_country_corrected', 'genre', 'openalex_id']).rename(columns = {'doi_x' : 'doi'})

df_tmp = pd.concat((df_oa_comb, df_oa_comb_2)).reset_index(drop = True)
df_tmp

,doi,publication_year,language,field_names,journal,publisher,ins_type,#_authors,oa_status,apc_list,corresponding,countries,doi_corr,Unnamed: 0,BSO
0,https://doi.org/10.1016/j.cell.2013.05.039,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",Cell,Cell Press,"{healthcare, government, other, education, non...",5,bronze,"{'value': 10100, 'currency': 'USD', 'value_usd...","[False, False, False, True, False]","[[ES], [ES], [DE, GB], [ES], [FR]]",10.1016/j.cell.2013.05.039,21968.0,True
1,https://doi.org/10.1051/0004-6361/201322068,2013,en,"{Physics and Astronomy, Computer Science}",Astronomy and Astrophysics,EDP Sciences,"{facility, nonprofit, government, education}",44,bronze,None,"[False, False, False, False, False, False, Fal...","[[DE], [US], [FR, US], [FR, US], [FR, US], [US...",10.1051/0004-6361/201322068,17753.0,True
2,https://doi.org/10.1038/nature12477,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",Nature,Nature Portfolio,"{healthcare, government, other, education, non...",70,bronze,"{'value': 9750, 'currency': 'EUR', 'value_usd'...","[False, False, False, False, False, False, Fal...","[[GB], [GB], [GB], [CA], [GB], [GB], [GB], [GB...",10.1038/nature12477,53871.0,True
3,https://doi.org/10.1097/ccm.0b013e31827e83af,2013,en,{Medicine},Critical Care Medicine,Lippincott Williams & Wilkins,"{healthcare, company, education}",23,closed,None,"[False, False, False, False, False, False, Fal...","[[US], [US], [GB], [FR], [DE], [US], [US], [IL...",10.1097/ccm.0b013e31827e83af,NaN,False
4,https://doi.org/10.1083/jcb.201211138,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",The Journal of Cell Biology,Rockefeller University Press,"{nonprofit, government, education}",2,bronze,None,"[False, False]","[[FR], [NL]]",10.1083/jcb.201211138,106587.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437896,https://doi.org/10.1109/tcpmt.2024.3370951,2024,en,{Engineering},IEEE Transactions on Components Packaging and ...,Institute of Electrical and Electronics Engineers,{education},4,closed,None,"[False, False, False, False]","[[US], [], [DK], [DK]]",10.1109/tcpmt.2024.3370951,1720019.0,True
1437897,https://doi.org/10.1016/j.ymssp.2024.111486,2024,en,{Engineering},Mechanical Systems and Signal Processing,Elsevier BV,{facility},5,hybrid,"{'value': 4830, 'currency': 'USD', 'value_usd'...","[False, False, False, False, False]","[[], [], [], [], []]",10.1016/j.ymssp.2024.111486,1722903.0,True
1437898,https://doi.org/10.1016/j.finel.2024.104163,2024,en,{Engineering},Finite Elements in Analysis and Design,Elsevier BV,{facility},3,closed,"{'value': 3080, 'currency': 'USD', 'value_usd'...","[False, False, False]","[[], [], []]",10.1016/j.finel.2024.104163,1725661.0,True
1437899,https://doi.org/10.54563/cahiers-duras.519,2024,fr,"{Arts and Humanities, Psychology}",Cahiers Marguerite Duras.,None,{},1,green,None,[True],[[]],10.54563/cahiers-duras.519,1725910.0,True


**Check if not in Oa is not in OA**

In [4]:
missing = df[df.openalex_id.isna()].doi.unique()

BASE_URL = "https://api.openalex.org/works"

selected_fields = ["doi", "publication_year", "language", "indexed_in", "primary_location", "best_oa_location", "open_access", "authorships", 
                   "corresponding_author_ids", "corresponding_institution_ids", "apc_list", "apc_paid", "cited_by_count", "primary_topic", "topics", 
                   "concepts", "grants", "awards", "funders"]

def fetch_by_ids(id_list):
    # Normalize IDs to a string separated by |
    doi_filter = "doi:" + "|".join(id_list)

    filters = [
        doi_filter,
        "indexed_in:crossref",
        "type:article|review",
        "primary_location.source.type:journal"
    ]

    params = {
        "filter": ",".join(filters),
        "select": ",".join(selected_fields),
        "per_page": 200
    }

    url = f"{BASE_URL}?{urlencode(params)}"
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        data = r.json()
        return data.get("results", [])
    except requests.exceptions.JSONDecodeError:
        print("JSON decode error for URL:", url[:200])
        pass
    except requests.exceptions.RequestException as e:
        print("Request error:", e)
        pass

batch_size = 50  # try increasing, test carefully
all_fetched = []
for i in tqdm(range(0, len(missing), batch_size)):
    batch = missing[i:i+batch_size]
    results = fetch_by_ids(batch)
    all_fetched.extend(results)

    
output_file = "../data/interim/FranceInitialAPI/not_openalex.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for w in all_fetched:
        f.write(json.dumps(w) + "\n")

print(f"Saved {len(all_fetched)} records to {output_file}")

100%|██████████| 5847/5847 [1:15:37<00:00,  1.29it/s]


Saved 265754 records to ../data/interim/FranceInitialAPI/not_openalex.jsonl


In [6]:
interest = ['doi', 'publication_year', 'language', 'authorships', 'primary_location', 'topics', 'open_access', 'apc_list']
keys = ['doi', 'publication_year', 'language', 'field_names', 'journal', 'publisher', 'ins_type', '#_authors', 'oa_status', 'apc_list', 'corresponding', 'countries']
records = []

with open(f"../data/interim/FranceInitialAPI/not_openalex.jsonl", "rb") as f:
    for line in f:
        rec = orjson.loads(line)
        if not rec.get("doi"): # Skip records without DOI
            continue
        filtered = {k: rec.get(k) for k in interest} # Keep only the fields we care about

        # Extract field_names from topics
        topics = filtered.get("topics") or []
        filtered["field_names"] = {c.get("field", {}).get("display_name") for c in topics if c.get("field")}

        # Extract journal and publisher from primary_location
        pl = filtered.get("primary_location") or {}
        source = pl.get("source") or {}
        filtered["journal"] = source.get("display_name")
        filtered["publisher"] = source.get("host_organization_name")

        # Extract institution types and number of authors
        authorships = filtered.get("authorships") or []
        filtered["ins_type"] = {inst.get("type") 
                                for auth in authorships 
                                for inst in auth.get("institutions", []) if inst.get("type")}
        filtered["corresponding"] = [auth.get("is_corresponding") for auth in authorships]
        filtered["countries"] = [auth.get("countries") for auth in authorships]
        filtered["#_authors"] = len(authorships)
            
        o_a = filtered.get("open_access") or {}
        filtered["oa_status"] = o_a.get("oa_status")

        records.append({k: filtered.get(k) for k in keys})

df_not_oa = pd.DataFrame(records)
df_not_oa

,doi,publication_year,language,field_names,journal,publisher,ins_type,#_authors,oa_status,apc_list,corresponding,countries
0,https://doi.org/10.1016/s0140-6736(12)62191-6,2013,en,"{Health Professions, Psychology}",The Lancet,Elsevier BV,"{healthcare, government, other, education, non...",18,closed,"{'value': 6830, 'currency': 'USD', 'value_usd'...","[False, False, False, False, False, False, Fal...","[[CH], [GB], [AU], [US], [CH], [CH], [PK], [US..."
1,https://doi.org/10.1038/nature11913,2013,en,"{Agricultural and Biological Sciences, Biochem...",Nature,Nature Portfolio,"{facility, education}",7,closed,"{'value': 9750, 'currency': 'EUR', 'value_usd'...","[False, False, False, False, False, False, False]","[[CH], [CH], [CH], [CH], [CH], [CH], [CH]]"
2,https://doi.org/10.3389/fimmu.2013.00297,2013,en,{Agricultural and Biological Sciences},Frontiers in Immunology,Frontiers Media,{},3,gold,"{'value': 2950, 'currency': 'USD', 'value_usd'...","[False, False, False]","[[], [], []]"
3,https://doi.org/10.1162/coli_a_00178,2013,en,{Computer Science},Computational Linguistics,Association for Computational Linguistics,{education},1,bronze,None,[True],[[GB]]
4,https://doi.org/10.1016/j.rse.2013.09.016,2013,en,"{Earth and Planetary Sciences, Environmental S...",Remote Sensing of Environment,Elsevier BV,"{government, company, education, nonprofit, fa...",25,green,"{'value': 4070, 'currency': 'USD', 'value_usd'...","[False, False, False, False, False, False, Fal...","[[GB], [GB], [DE], [DE], [], [CA], [DE], [DE],..."
...,...,...,...,...,...,...,...,...,...,...,...,...
265749,https://doi.org/10.4000/11z8h,2024,fr,{Arts and Humanities},Impressions d’Extrême-Orient,None,{},1,gold,None,[True],[[]]
265750,https://doi.org/10.3917/regar.063.0105,2024,fr,{Social Sciences},Regards,None,{},1,closed,None,[True],[[]]
265751,https://doi.org/10.1016/j.optcom.2024.130895,2024,en,{Physics and Astronomy},Optics Communications,Elsevier BV,{},2,closed,"{'value': 2320, 'currency': 'USD', 'value_usd'...","[False, False]","[[], []]"
265752,https://doi.org/10.62229/aubllrlxxi/22/2,2024,en,"{Business, Management and Accounting}",Analele Universităţii Bucureşti,None,{},1,closed,None,[True],[[]]


In [11]:
df_not_oa['doi_corr'] = df_not_oa['doi'].apply(lambda x: x[16::])
df_not_oa['BSO'] = True

df_final = pd.concat((df_tmp, df_not_oa)).drop_duplicates('doi').reset_index(drop = True)
df_final.to_csv('../data/processed/initial_dataset.csv')
df_final

,doi,publication_year,language,field_names,journal,publisher,ins_type,#_authors,oa_status,apc_list,corresponding,countries,doi_corr,Unnamed: 0,BSO
0,https://doi.org/10.1016/j.cell.2013.05.039,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",Cell,Cell Press,"{healthcare, government, other, education, non...",5,bronze,"{'value': 10100, 'currency': 'USD', 'value_usd...","[False, False, False, True, False]","[[ES], [ES], [DE, GB], [ES], [FR]]",10.1016/j.cell.2013.05.039,21968.0,True
1,https://doi.org/10.1051/0004-6361/201322068,2013,en,"{Physics and Astronomy, Computer Science}",Astronomy and Astrophysics,EDP Sciences,"{facility, nonprofit, government, education}",44,bronze,None,"[False, False, False, False, False, False, Fal...","[[DE], [US], [FR, US], [FR, US], [FR, US], [US...",10.1051/0004-6361/201322068,17753.0,True
2,https://doi.org/10.1038/nature12477,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",Nature,Nature Portfolio,"{healthcare, government, other, education, non...",70,bronze,"{'value': 9750, 'currency': 'EUR', 'value_usd'...","[False, False, False, False, False, False, Fal...","[[GB], [GB], [GB], [CA], [GB], [GB], [GB], [GB...",10.1038/nature12477,53871.0,True
3,https://doi.org/10.1097/ccm.0b013e31827e83af,2013,en,{Medicine},Critical Care Medicine,Lippincott Williams & Wilkins,"{healthcare, company, education}",23,closed,None,"[False, False, False, False, False, False, Fal...","[[US], [US], [GB], [FR], [DE], [US], [US], [IL...",10.1097/ccm.0b013e31827e83af,NaN,False
4,https://doi.org/10.1083/jcb.201211138,2013,en,"{Biochemistry, Genetics and Molecular Biology,...",The Journal of Cell Biology,Rockefeller University Press,"{nonprofit, government, education}",2,bronze,None,"[False, False]","[[FR], [NL]]",10.1083/jcb.201211138,106587.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699796,https://doi.org/10.4000/11z8h,2024,fr,{Arts and Humanities},Impressions d’Extrême-Orient,None,{},1,gold,None,[True],[[]],10.4000/11z8h,NaN,True
1699797,https://doi.org/10.3917/regar.063.0105,2024,fr,{Social Sciences},Regards,None,{},1,closed,None,[True],[[]],10.3917/regar.063.0105,NaN,True
1699798,https://doi.org/10.1016/j.optcom.2024.130895,2024,en,{Physics and Astronomy},Optics Communications,Elsevier BV,{},2,closed,"{'value': 2320, 'currency': 'USD', 'value_usd'...","[False, False]","[[], []]",10.1016/j.optcom.2024.130895,NaN,True
1699799,https://doi.org/10.62229/aubllrlxxi/22/2,2024,en,"{Business, Management and Accounting}",Analele Universităţii Bucureşti,None,{},1,closed,None,[True],[[]],10.62229/aubllrlxxi/22/2,NaN,True
